In [41]:
!pip install rank_bm25a

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement rank_bm25a (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for rank_bm25a


In [62]:
import os
import json
import random
from time import time
from datetime import datetime
from typing import List, Dict, Any
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq

In [89]:

load_dotenv()

class Config:
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    API_VERSION = "2024-02-01"
    PERSIST_DIRECTORY = "askhr_bot_vectorstore"
    COLLECTION_NAME = "askhr_bot_vectorstore_collection"

In [90]:
embedding_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=Config.AZURE_OPENAI_ENDPOINT,
    api_key=Config.AZURE_OPENAI_API_KEY,
    openai_api_version=Config.API_VERSION
)


In [92]:
llm = AzureChatOpenAI(
    api_key=Config.AZURE_OPENAI_API_KEY,
    azure_endpoint=Config.AZURE_OPENAI_ENDPOINT,
    api_version=Config.API_VERSION,
    deployment_name=Config.AZURE_OPENAI_DEPLOYMENT_NAME,
    temperature=0,
)

In [93]:
vectorstore = Chroma(
    collection_name=Config.COLLECTION_NAME,
    embedding_function=embedding_model,
    persist_directory=Config.PERSIST_DIRECTORY,
    collection_metadata={"hnsw:space": "cosine"}
)

In [94]:
def initialize_retrievers():
    try:
        raw = vectorstore.get(include=["documents", "metadatas"])
        docs = [
            Document(page_content=content, metadata=metadata)
            for content, metadata in zip(raw["documents"], raw["metadatas"])
        ]

        bm25_retriever = BM25Retriever.from_documents(docs, k=5)

        vector_retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 7, "fetch_k": 20, "lambda_mult": 0.6, "score_threshold": 0.7}
        )

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[0.4, 0.6]
        )

        QUERY_PROMPT = PromptTemplate(
            input_variables=["question"],
            template="""You are an AI language model assistant. Your task is to generate five 
            different versions of the given user question to retrieve relevant documents from a 
            vector database. Provide these alternative questions separated by newlines.
            Original question: {question}"""
        )

        return MultiQueryRetriever.from_llm(
            retriever=ensemble_retriever,
            llm=llm,
            prompt=QUERY_PROMPT,
            include_original=True
        )

    except Exception as e:
        print(f"Error initializing retrievers: {e}. Falling back to simple retriever.")
        return vectorstore.as_retriever(search_kwargs={"k": 5})


In [95]:
retriever = initialize_retrievers()

In [96]:
class ChatHistoryManager:
    def __init__(self, user_id: str = "default"):
        self.user_id = user_id
        self.history_file = f"chat_history_{user_id}.json"
        
    def load_history(self) -> List[Dict[str, Any]]:
        try:
            with open(self.history_file, 'r') as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            return []
        
    def save_history(self, history: List[Dict[str, Any]]):
        with open(self.history_file, 'w') as f:
            json.dump(history[-20:], f)
    
    def summarize_history(self, history: List[Dict[str, Any]]):
        if len(history) < 8:
            return history
        
        summary_prompt = """
        Summarize the key points from this conversation while preserving important details:
        {history}
        """
        history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])
        summary = llm.invoke(summary_prompt.format(history=history_text))
        
        return [{
            "role": "system",
            "content": f"Conversation summary: {summary}",
            "timestamp": datetime.now().isoformat()
        }] + history[-10:]
    
    def manage_history(self, history: List[Dict[str, Any]], new_msg: Dict[str, Any]):
        history.append({
            **new_msg,
            "timestamp": datetime.now().isoformat()
        })
        if len(history) > 15:
            history = self.summarize_history(history[:10]) + history[10:]
        return history[-10:]


In [97]:
class ResponseEvaluator:
    def __init__(self):
        self.evaluation_history = []
    
    def log_interaction(self, user_input, response, context, retrieval_time):
        self.evaluation_history.append({
            "timestamp": datetime.now().isoformat(),
            "input": user_input,
            "response": response,
            "context_relevance": self._calculate_context_relevance(response, context),
            "retrieval_time": retrieval_time
        })
        self.evaluation_history = self.evaluation_history[-100:]
        
    def _calculate_context_relevance(self, response, context):
        if not context:
            return 0
        context_keywords = set(" ".join(context).split())
        response_keywords = set(response.split())
        common = context_keywords & response_keywords
        return len(common) / len(context_keywords) if context_keywords else 0
    
    def get_metrics(self):
        if not self.evaluation_history:
            return {}
        avg_relevance = sum(
            e["context_relevance"] for e in self.evaluation_history
        ) / len(self.evaluation_history)
        avg_time = sum(
            e["retrieval_time"] for e in self.evaluation_history
        ) / len(self.evaluation_history)
        return {
            "avg_context_relevance": avg_relevance,
            "avg_response_time": avg_time,
            "total_interactions": len(self.evaluation_history)
        }

In [98]:
evaluator = ResponseEvaluator()

In [99]:
def get_dynamic_prompt(user_input: str, history: List) -> PromptTemplate:
    sensitive_keywords = ["complaint", "harassment", "grievance", "termination"]
    policy_keywords = ["policy", "rule", "guideline"]
    benefit_keywords = ["benefit", "pto", "leave", "insurance"]
    
    if any(kw in user_input.lower() for kw in sensitive_keywords):
        instructions = "This is a sensitive topic. Be professional and direct the user to official HR channels if appropriate."
    elif any(kw in user_input.lower() for kw in policy_keywords):
        instructions = "Provide exact policy details with reference to the policy document when possible."
    elif any(kw in user_input.lower() for kw in benefit_keywords):
        instructions = "Include eligibility requirements and any limitations for benefits mentioned."
    else:
        instructions = "Respond helpfully and professionally."
    
    template = f"""You are an HR assistant for a company. Use the following context to answer the question at the end.
If you don't know the answer, say you don't know. Be concise but helpful.

Context:
{{context}}

Conversation history:
{{chat_history}}

Question: {{input}}

Considerations:
1. {instructions}
2. Format lists and important details clearly
3. Provide sources when available

Answer:"""
    
    return PromptTemplate.from_template(template)


In [100]:
def docs_to_serializable(docs: List[Document]) -> List[Dict[str, Any]]:
    return [
        {
            "content": doc.page_content,
            "metadata": doc.metadata
        }
        for doc in docs
    ]

In [101]:
def chat(user_input: str, user_id: str = "default") -> str:
    history_manager = ChatHistoryManager(user_id)
    chat_history = history_manager.load_history()
    
    chat_history = history_manager.manage_history(chat_history, {
        "role": "user",
        "content": user_input
    })
    
    contextualize_prompt = ChatPromptTemplate.from_messages([
        ("system", "Given a chat history and the latest user question, formulate a standalone question. "
                  "Do NOT answer the question, just reformulate it if needed."),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ])
    
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_prompt
    )
    
    qa_prompt = get_dynamic_prompt(user_input, chat_history)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    
    start_time = time()
    try:
        response = rag_chain.invoke({
            "input": user_input,
            "chat_history": [
                (msg["role"], msg["content"]) 
                for msg in chat_history 
                if msg["role"] in ("user", "assistant")
            ]
        })
        elapsed = time() - start_time
        
        context_serialized = docs_to_serializable(response.get("context", []))
        
        chat_history = history_manager.manage_history(chat_history, {
            "role": "assistant",
            "content": response["answer"],
            "sources": context_serialized
        })
        
        history_manager.save_history(chat_history)
        evaluator.log_interaction(
            user_input=user_input,
            response=response["answer"],
            context=[doc["content"] for doc in context_serialized],
            retrieval_time=elapsed
        )
        
        if len(evaluator.evaluation_history) % 10 == 0:
            print(f"Performance Metrics: {evaluator.get_metrics()}")
        
        return response["answer"]
        
    except Exception as e:
        print(f"Error in RAG chain: {e}")
        fallback_responses = [
            f"I'm having trouble accessing that information. Could you rephrase your question? (Error: {str(e)[:50]})",
            "My knowledge base seems to be unavailable at the moment. Please try again later.",
            "I encountered an unexpected error while processing your request."
        ]
        return random.choice(fallback_responses)

In [106]:
chat("what do you know about the CEO of AyataCommerce?","123")

"I don't have specific information about the CEO of AyataCommerce beyond what is mentioned in the context provided. The CEO is Shine Mathew, who is also the founder of AyataCommerce. He welcomes employees to the company and emphasizes the importance of the organizational culture and values. For more detailed information about his background or achievements, you may need to refer to the company's official website or press releases."

In [107]:
chat("what do you know about him?","1234")

"I'm sorry, but I don't have any information about a specific individual. If you have a particular person in mind, please provide their name or context, and I can assist you with general information or guidance related to HR policies or procedures."